In [1]:
#hide
#default_exp experiment_manager
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Experiment Manager

> Main class

In [2]:
#export
# coding: utf-8
import pickle
import sys
import os
import numpy as np
import pandas as pd
import time
import datetime
from sklearn.model_selection import ParameterGrid
from sklearn.utils import Bunch
import platform
import pprint
import subprocess
import json
from multiprocessing import Process
import logging
import traceback
import shutil
from fastcore.utils import store_attr

# hpsearch core API
from hpsearch.config.manager_factory import ManagerFactory
from hpsearch.utils import experiment_utils
from hpsearch.utils.experiment_utils import remove_defaults
from hpsearch.utils.organize_experiments import remove_defaults_from_experiment_data
import hpsearch.config.hp_defaults as dflt

In [3]:
#for tests
import pytest
import os

from block_types.utils.nbdev_utils import md

from hpsearch.examples.complex_dummy_experiment_manager import ComplexDummyExperimentManager

## ExperimentManager

In [4]:
#export
class ExperimentManager (object):

    def __init__ (self, 
                  allow_base_class=dflt.allow_base_class,
                  path_experiments=dflt.path_experiments,
                  defaults=dflt.defaults,
                  root=dflt.root,
                  metric=dflt.metric,
                  op=dflt.op,
                  path_alternative=None,
                  path_data=None,
                  name_model_history=dflt.name_model_history,
                  model_file_name=dflt.model_file_name,
                  name_epoch=dflt.name_epoch,
                  result_file=dflt.result_file):
        
        #store_attr ()
        if True:
            self.allow_base_class = allow_base_class
            self.path_experiments = path_experiments
            self.defaults = defaults
            self.key_score = metric
            self.path_alternative = path_alternative
            self.path_data = path_data
            self.name_model_history = name_model_history
            self.model_file_name = model_file_name
            self.name_epoch = name_epoch
            self.result_file = result_file
        
        self.key_score = metric
        self.parameters_non_pickable = {}
        self.default_operations = dict(root=root,
                                       metric=metric,
                                       op=op)
        self.manager_factory = ManagerFactory(allow_base_class=allow_base_class)
        self.manager_factory.register_manager (self)

    def get_default_parameters (self, parameters):
        if not self.allow_base_class:
            raise ImportError ('call get_default_parameters from base class is not allowed')
        return self.defaults
    
    def get_default_operations (self):
        return self.default_operations

    def get_path_experiments (self, path_experiments = None, folder = None):
        """Gives the root path to the folder where results of experiments are stored."""

        path_experiments = (path_experiments if path_experiments is not None 
                            else self.path_experiments)
        if folder != None: path_experiments = f'{path_experiments}/{folder}'

        return path_experiments

    def get_path_alternative (self, path_results):
        if self.path_alternative is None:
            path_alternative = path_results

        return path_alternative

    def get_path_data (self, run_number, root_path=None, parameters={}):
        if self.path_data is None:
            if root_path is None:
                root_path = self.get_path_experiments()
            return f'{root_path}/data'
        else:
            return self.path_data

    def get_path_experiment (self, experiment_id, root_path=None, root_folder=None):
        if root_path is None:
            root_path = self.get_path_experiments(folder=root_folder)
        path_experiment = f'%s/experiments/%05d' %(root_path,experiment_id)
        return path_experiment

    def get_path_results (self, experiment_id, run_number, root_path=None, root_folder=None):
        path_experiment = self.get_path_experiment (experiment_id, root_path=root_path, root_folder=root_folder)
        path_results = '%s/%d' %(path_experiment,run_number)
        return path_results
    
    def get_experiment_data (self, path_experiments=None, folder_experiments=None, experiments=None):
        path_experiments = self.get_path_experiments(path_experiments=path_experiments, 
                                                    folder=folder_experiments)
        path_csv = '%s/experiments_data.csv' %path_experiments
        path_pickle = path_csv.replace('csv', 'pk')
        if os.path.exists (path_pickle):
            experiment_data = pd.read_pickle (path_pickle)
        else:
            experiment_data = pd.read_csv (path_csv, index_col=0)
        if experiments is not None:
            experiment_data = experiment_data.loc[experiments,:]
            
        return experiment_data
    
    def get_key_score (self, other_parameters):
        key_score = other_parameters.get('key_score')
        suffix_results = other_parameters.get('suffix_results', '')
        if key_score is None and (len(suffix_results) > 0):
            if suffix_results[0] == '_':
                key_score = suffix_results[1:]
            else:
                key_score = suffix_results
        key_score = self.key_score if key_score is None else key_score
        
        return key_score
    
    def get_name_epoch (self, other_parameters):
        return other_parameters.get ('name_epoch', self.name_epoch)
    
    def remove_previous_experiments (self, path_experiments = None, folder = None):
        path_experiments = self.get_path_experiments (path_experiments=path_experiments, 
                                                      folder=folder)
        if os.path.exists (path_experiments):
            shutil.rmtree (path_experiments)

    def experiment_visualization (self, **kwargs):
        raise ValueError ('this type of experiment visualization is not recognized')

    def run_experiment_pipeline (self, run_number=0, path_results='./results', parameters = {}):
        """ Runs complete learning pipeline: loading / generating data, building and learning model, applying it to data,
        and evaluating it."""
        start_time = time.time()

        # record all parameters except for non-pickable ones
        record_parameters (path_results, parameters)

        # integrate non-pickable parameters into global dictionary
        parameters.update (self.parameters_non_pickable)
        self.parameters_non_pickable = {}

        logger = logging.getLogger("experiment_manager")

        # #####################################
        # Evaluation
        # #####################################
        time_before = time.time()
        score_dict = self._run_experiment (parameters=parameters, path_results=path_results, run_number=run_number)
        logger.info ('time spent on this experiment: {}'.format(time.time()-time_before))

        # #####################################
        # Final scores
        # #####################################
        score_name = parameters.get('suffix_results','')
        if len(score_name) > 0:
            if score_name[0] == '_':
                score_name = score_name[1:]
            if score_dict.get(score_name) is not None:
                logger.info (f'score: {score_dict.get(score_name)}')

        spent_time = time.time() - time_before

        return score_dict, spent_time

    # *************************************************************************
    #   run_experiment methods
    # *************************************************************************
    def _run_experiment (self, parameters={}, path_results='./results', run_number=None):

        parameters['run_number'] = run_number

        # wrap parameters
        parameters = Bunch(**parameters)

        if parameters.get('use_process', False):
            return self.run_experiment_in_separate_process (parameters, path_results)
        else:
            return self.run_experiment (parameters=parameters, path_results=path_results)

    def run_experiment_in_separate_process (self, parameters={}, path_results='./results'):

        parameters['return_results']=False
        p = Process(target=self.run_experiment_saving_results, args=(parameters, path_results))
        p.start()
        p.join()

        dict_results = pickle.load (open ('%s/dict_results.pk' %path_results, 'rb'))

        return dict_results

    def run_experiment_saving_results (self, parameters={}, path_results='./results'):
        dict_results = self.run_experiment (parameters=parameters, path_results=path_results)
        pickle.dump (dict_results, open ('%s/dict_results.pk' %path_results, 'wb'))

    def run_experiment (self, parameters={}, path_results='./results'):
        raise NotImplementedError ('This method needs to be defined in subclass')


    # *************************************************************************
    # *************************************************************************
    def create_experiment_and_run (self, parameters = {}, other_parameters = {}, root_path=None, 
                                   run_number=0, log_message=None):
        """
        
        """

        # ****************************************************
        #  preliminary set-up: logger and root_path
        # ****************************************************
        logger = logging.getLogger("experiment_manager")
        if log_message is not None:
            logger.info ('**************************************************')
            logger.info (log_message)
            logger.info ('**************************************************')
            other_parameters['log_message'] = log_message

        # insert path to experiment script file that called the experiment manager
        other_parameters = other_parameters.copy()
        insert_experiment_script_path (other_parameters, logger)

        # get root_path and create directories
        if root_path is None:
            root_path = self.get_path_experiments(folder = other_parameters.get('root_folder'))
        os.makedirs (root_path, exist_ok = True)

        # ****************************************************
        # register (subclassed) manager so that it can be used by decoupled modules
        # ****************************************************
        self.register_and_store_subclassed_manager ()

        # ****************************************************
        #   get experiment number given parameters
        # ****************************************************
        parameters = remove_defaults (parameters)

        path_csv = '%s/experiments_data.csv' %root_path
        path_pickle = path_csv.replace('csv', 'pk')
        experiment_number, experiment_data = load_or_create_experiment_values (
            path_csv, parameters, precision=other_parameters.get('precision', 1e-15)
        )

        #save_other_parameters (experiment_number, other_parameters, root_path)

        # if old experiment, we can require that given parameters match with experiment number
        if (other_parameters.get('experiment_number') is not None 
            and experiment_number != other_parameters.get('experiment_number')):
            raise ValueError (f'expected number: {other_parameters.get("experiment_number")}, '
                              f'found: {experiment_number}')
        other_parameters['experiment_number'] = experiment_number

        # ****************************************************
        # get key_score and suffix_results
        # ****************************************************
        key_score = self.get_key_score (other_parameters)
        if key_score is not None:
            suffix_results = f'_{key_score}'
            other_parameters['suffix_results'] = suffix_results

        # ****************************************************
        #   get run_id, if not given
        # ****************************************************
        if run_number is None:
            run_number = 0
            name_score = '%d%s' %(run_number, suffix_results)
            while not isnull(experiment_data, experiment_number, name_score):
                logger.info ('found previous run for experiment number {}, run {}, with score {} = {}'.format(experiment_number, run_number, key_score, experiment_data.loc[experiment_number, name_score]))
                run_number += 1
                name_score = '%d%s' %(run_number, suffix_results)
            logger.info ('starting experiment {} with run number {}'.format(experiment_number, run_number))

        else:
            name_score = '%d%s' %(run_number, suffix_results)
            if not isnull(experiment_data, experiment_number, name_score):
                previous_result = experiment_data.loc[experiment_number, name_score]
                logger.info ('found completed: experiment number: %d, run number: %d - score: %f' %(experiment_number, run_number, previous_result))
                logger.info (parameters)
                if other_parameters.get('repeat_experiment', False):
                    logger.info ('redoing experiment')

        # ****************************************************
        #   remove unfinished experiments
        # ****************************************************
        if other_parameters.get('remove_not_finished', False):
            name_finished = '%d_finished' %run_number
            if not isnull(experiment_data, experiment_number, name_finished):
                finished = experiment_data.loc[experiment_number, name_finished]
                logger.info (f'experiment {experiment_number}, run number {run_number}, finished {finished}')
                if not finished:
                    experiment_data.loc[experiment_number, name_score] = None
                    experiment_data.to_csv (path_csv)
                    experiment_data.to_pickle (path_pickle)
                    logger.info (f'removed experiment {experiment_number}, '
                                 f'run number {run_number}, finished {finished}')
            if other_parameters.get('only_remove_not_finished', False):
                return None, {}

        unfinished_flag = False
        name_epoch = self.get_name_epoch(other_parameters)
        current_path_results = self.get_path_results (experiment_number, run_number=run_number, 
                                                      root_path=root_path)

        # ****************************************************
        #   check conditions for skipping experiment
        # ****************************************************
        if (not isnull(experiment_data, experiment_number, name_score) 
            and not other_parameters.get('repeat_experiment', False)):
            if (other_parameters.get('check_finished', False) 
                and not self.finished_all_epochs (parameters, current_path_results, name_epoch)):
                unfinished_flag = True
            else:
                logger.info ('skipping...')
                return previous_result, {key_score: previous_result}
        elif (isnull(experiment_data, experiment_number, name_score) 
              and other_parameters.get('recompute_metrics', False) 
              and not other_parameters.get('force_recompute_metrics', False)):
            logger.info (f'experiment not found, skipping {run_number} due to only recompute_metrics')
            return None, {}
        
        # ****************************************************
        # log info
        # ****************************************************
        logger.info ('running experiment %d' %experiment_number)
        logger.info ('run number: %d' %run_number)
        logger.info ('\nparameters:\n%s' %mypprint(parameters))

        # ****************************************************
        #  get paths
        # ****************************************************
        # path_root_experiment folder
        path_root_experiment = '%s/experiments/%05d' %(root_path,experiment_number)
        mymakedirs(path_root_experiment, exist_ok=True)

        # path_experiment folder (where results are)
        path_experiment = '%s/%d' %(path_root_experiment, run_number)
        mymakedirs(path_experiment, exist_ok=True)

        # path to save big files
        path_experiment_big_size = self.get_path_alternative (path_experiment)
        os.makedirs (path_experiment_big_size, exist_ok = True)
        other_parameters['path_results_big'] = path_experiment_big_size

        # ****************************************************
        # get git and record parameters
        # ****************************************************
        # get git revision number
        other_parameters['git_hash'] = get_git_revision_hash(root_path)

        # write parameters in root experiment folder
        record_parameters (path_root_experiment, parameters, other_parameters)

        # store hyper_parameters in dictionary that maps experiment_number with hyper_parameter values
        store_parameters (root_path, experiment_number, parameters)

        # ****************************************************************
        # loggers
        # ****************************************************************
        logger_experiment = set_logger ("experiment", path_experiment)
        logger_experiment.info ('script: {}, line number: {}'.format(other_parameters['script_path'], other_parameters['lineno']))
        if os.path.exists(other_parameters['script_path']):
            shutil.copy (other_parameters['script_path'], path_experiment)
            shutil.copy (other_parameters['script_path'], path_root_experiment)

        # summary logger
        logger_summary = set_logger ("summary", root_path, mode='w', stdout=False, just_message=True, filename='summary.txt')
        logger_summary.info ('\n\n{}\nexperiment: {}, run: {}\nscript: {}, line number: {}\nparameters:\n{}{}'.format('*'*100, experiment_number, run_number, other_parameters['script_path'], other_parameters['lineno'], mypprint(parameters), '*'*100))
        if other_parameters.get('rerun_script') is not None:
            logger_summary.info ('\nre-run:\n{}'.format(other_parameters['rerun_script']))
        # same file in path_experiments
        logger_summary2 = set_logger ("summary", path_experiment, mode='w', stdout=False, just_message=True, filename='summary.txt')
        logger_summary2.info ('\n\n{}\nexperiment: {}, run: {}\nscript: {}, line number: {}\nparameters:\n{}{}'.format('*'*100, experiment_number, run_number, other_parameters['script_path'], other_parameters['lineno'], mypprint(parameters), '*'*100))

        # ****************************************************************
        # Do final adjustments to parameters
        # ****************************************************************
        parameters = parameters.copy()
        original_parameters = parameters.copy()
        parameters.update(other_parameters)

        # add default parameters - their values are overwritten by input values, if given
        defaults = self.get_default_parameters(parameters)
        parameters_with_defaults = defaults.copy()
        parameters_with_defaults.update(parameters)
        parameters = parameters_with_defaults

        # ***********************************************************
        # resume from previous experiment 
        # ***********************************************************
        if (isnull(experiment_data, experiment_number, name_score) 
            and other_parameters.get('check_finished_if_interrupted', False)
            and not self.finished_all_epochs (parameters, current_path_results, name_epoch)):
            unfinished_flag = True
        
        resuming_from_prev_epoch_flag = False
        if parameters.get('prev_epoch', False):
            logger.info('trying prev_epoch')
            experiment_data2 = experiment_data.copy()
            if (not unfinished_flag 
                and (other_parameters.get('repeat_experiment', False) 
                     or isnull(experiment_data, experiment_number, name_score))):
                    experiment_data2 = experiment_data2.drop(experiment_number,axis=0)
            prev_experiment_number = self.find_closest_epoch (experiment_data2, original_parameters, 
                                                              name_epoch=name_epoch)
            if prev_experiment_number is not None:
                logger.info(f'using prev_epoch: {prev_experiment_number}')
                prev_path_results = self.get_path_results (prev_experiment_number, 
                                                           run_number=run_number, 
                                                           root_path=root_path)
                found = self.make_resume_from_checkpoint (parameters, prev_path_results)
                if found:
                    logger.info (f'found previous exp: {prev_experiment_number}')
                    if prev_experiment_number == experiment_number:
                        if 'use_previous_best' not in other_parameters:
                            other_parameters['use_previous_best'] = parameters.get('use_previous_best', 
                                                                                   dflt.use_previous_best)
                        if not other_parameters['use_previous_best'] and unfinished_flag:
                            prev_epoch = self.get_last_epoch (parameters, 
                                                              current_path_results, 
                                                              name_epoch)
                            prev_epoch = max (int(prev_epoch), 0)
                            parameters[name_epoch] = parameters[name_epoch] - prev_epoch
                        logger.info ('using previous best')
                    else:
                        prev_epoch = experiment_data.loc[prev_experiment_number,name_epoch]
                        prev_epoch = (int(prev_epoch) if prev_epoch is not None 
                                      else defaults.get(name_epoch))
                        parameters[name_epoch] = parameters[name_epoch] - prev_epoch

                resuming_from_prev_epoch_flag = found
                

        if not resuming_from_prev_epoch_flag and parameters.get('from_exp', None) is not None:
            prev_experiment_number = parameters.get('from_exp', None)
            logger.info('using previous experiment %d' %prev_experiment_number)
            prev_path_results = self.get_path_results (prev_experiment_number, run_number=run_number, 
                                                       root_path=root_path)
            self.make_resume_from_checkpoint (parameters, prev_path_results, use_best=True)

        # ****************************************************************
        #   Analyze if experiment was interrupted
        # ****************************************************************
        if parameters.get('skip_interrupted', False):
            was_interrumpted = self.exists_current_checkpoint (parameters, path_experiment)
            was_interrumpted = (was_interrumpted or 
                                self.obtain_last_result (parameters, path_experiment) is not None)
            if was_interrumpted:
                logger.info ('found intermediate results, skipping...')
                return None, {}

        # ****************************************************************
        # retrieve last results in interrupted experiments
        # ****************************************************************
        run_pipeline = True
        if parameters.get('use_last_result', False):
            experiment_result = self.obtain_last_result (parameters, path_experiment)
            if experiment_result is None and parameters.get('run_if_not_interrumpted', False):
                run_pipeline = True
            elif experiment_result is None:
                return None, {}
            else:
                run_pipeline = False

        # ****************************************************************
        # run experiment
        # ****************************************************************
        if run_pipeline:
            experiment_result, time_spent = self.run_experiment_pipeline (run_number,
                                        path_experiment,
                                        parameters=parameters)
            finished = True
        else:
            finished = False

        # ****************************************************************
        #  Retrieve and store results
        # ****************************************************************
        if type(experiment_result)==dict:
            dict_results = experiment_result
            for key in dict_results.keys():
                if key != '':
                    experiment_data.loc[experiment_number, '%d_%s' %(run_number, key)]=dict_results[key]
                else:
                    experiment_data.loc[experiment_number, '%d' %run_number]=dict_results[key]
                logger.info('{} - {}: {}'.format(run_number, key, dict_results[key]))
        else:
            experiment_data.loc[experiment_number, name_score]=experiment_result
            logger.info('{} - {}: {}'.format(run_number, name_score, experiment_result))
            dict_results = {name_score:experiment_result}

        if isnull(experiment_data, experiment_number, 'time_'+str(run_number)) and finished:
            experiment_data.loc[experiment_number,'time_'+str(run_number)]=time_spent
        experiment_data.loc[experiment_number, 'date']=datetime.datetime.time(datetime.datetime.now())
        experiment_data.loc[experiment_number, '%d_finished' %run_number]=finished

        experiment_data.to_csv(path_csv)
        experiment_data.to_pickle(path_pickle)

        save_other_parameters (experiment_number, other_parameters, root_path)

        logger_summary2.info ('\nresults:\n{}'.format(dict_results))
        logger.info ('finished experiment %d' %experiment_number)

        # return final score
        result = dict_results.get(name_score)
        return result, dict_results

    def grid_search (self, parameters_multiple_values={}, parameters_single_value={}, other_parameters = {},
                     root_path=None, run_numbers=[0], random_search=False,
                     load_previous=False, log_message='', nruns = None, keep='multiple'):

        other_parameters = other_parameters.copy()

        os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
        if nruns is not None:
            run_numbers = range (nruns)

        if root_path is None:
            root_path = self.get_path_experiments(folder  = other_parameters.get('root_folder'))
        path_results_base = root_path

        mymakedirs(path_results_base,exist_ok=True)

        if keep=='multiple':
            parameters_single_value = {k:parameters_single_value[k] for k in parameters_single_value.keys() if k not in parameters_multiple_values}
        elif keep=='single':
            parameters_multiple_values = {k:parameters_multiple_values[k] for k in parameters_multiple_values.keys() if k not in parameters_single_value}
        else:
            raise ValueError ('parameter keep {} not recognized: it must be either multiple or single'.format(keep))

        parameters_multiple_values_all = parameters_multiple_values
        parameters_multiple_values_all = list(ParameterGrid(parameters_multiple_values_all))

        logger = set_logger ("experiment_manager", path_results_base)
        if log_message != '':
            other_parameters['log_message'] = log_message
        insert_experiment_script_path (other_parameters, logger)

        if random_search:
            path_random_hp = '%s/random_hp.pk' %path_results_base
            if load_previous and os.path.exists(path_random_hp):
                parameters_multiple_values_all = pickle.load(open(path_random_hp,'rb'))
            else:
                parameters_multiple_values_all = list(np.random.permutation(parameters_multiple_values_all))
                pickle.dump (parameters_multiple_values_all, open(path_random_hp,'wb'))
        for (i_hp, parameters_multiple_values) in enumerate(parameters_multiple_values_all):
            parameters = parameters_multiple_values.copy()
            parameters.update(parameters_single_value)

            for (i_run, run_number) in enumerate(run_numbers):
                logger.info('processing hyper-parameter %d out of %d' %(i_hp, len(parameters_multiple_values_all)))
                logger.info('doing run %d out of %d' %(i_run, len(run_numbers)))
                logger.info('%s' %log_message)

                self.create_experiment_and_run (parameters=parameters, other_parameters = other_parameters,
                                           run_number=run_number, root_path=path_results_base)

        # This solves an intermitent issue found in TensorFlow (reported as bug by community)
        import gc
        gc.collect()

    def run_multiple_repetitions (self, parameters={}, other_parameters = {},
                     root_path=None, log_message='', nruns = None, run_numbers=[0]):

        other_parameters = other_parameters.copy()

        if nruns is not None:
            run_numbers = range (nruns)

        logger = set_logger ("experiment_manager", root_path)
        results = np.zeros((len(run_numbers),))
        for (i_run, run_number) in enumerate(run_numbers):
                logger.info('doing run %d out of %d' %(i_run, len(run_numbers)))
                logger.info('%s' %log_message)

                results[i_run], dict_results  = self.create_experiment_and_run (parameters=parameters, other_parameters = other_parameters,
                                           run_number=run_number, root_path=root_path)
                if dict_results.get('is_pruned', False):
                    break

        mu, std = results.mean(), results.std()
        logger.info ('mean {}: {}, std: {}'.format(other_parameters.get('key_score',''), mu, std))

        dict_results[other_parameters.get('key_score','cost')] = mu

        return mu, std, dict_results


    def hp_optimization (self, parameter_sampler = None, root_path=None, log_message=None, parameters={}, other_parameters={}, nruns=None):

        import optuna
        from optuna.pruners import SuccessiveHalvingPruner, MedianPruner
        from optuna.samplers import RandomSampler, TPESampler
        from optuna.integration.skopt import SkoptSampler

        if root_path is None:
            root_path = self.get_path_experiments(folder  = other_parameters.get('root_folder'))

        other_parameters = other_parameters.copy()

        os.makedirs(root_path, exist_ok=True)
        logger = set_logger ("experiment_manager", root_path)
        if log_message != '':
            other_parameters['log_message'] = log_message
        insert_experiment_script_path (other_parameters, logger)

        # n_warmup_steps: Disable pruner until the trial reaches the given number of step.
        sampler_method = other_parameters.get('sampler_method', 'random')
        pruner_method = other_parameters.get('pruner_method', 'halving')
        n_evaluations = other_parameters.get('n_evaluations', 20)
        seed = other_parameters.get('seed', 0)
        if sampler_method == 'random':
            sampler = RandomSampler(seed=seed)
        elif sampler_method == 'tpe':
            sampler = TPESampler(n_startup_trials=other_parameters.get('n_startup_trials', 5), seed=seed)
        elif sampler_method == 'skopt':
            # cf https://scikit-optimize.github.io/#skopt.Optimizer
            # GP: gaussian process
            # Gradient boosted regression: GBRT
            sampler = SkoptSampler(skopt_kwargs={'base_estimator': "GP", 'acq_func': 'gp_hedge'})
        else:
            raise ValueError('Unknown sampler: {}'.format(sampler_method))

        if pruner_method == 'halving':
            pruner = SuccessiveHalvingPruner(min_resource=1, reduction_factor=4, min_early_stopping_rate=0)
        elif pruner_method == 'median':
            pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=n_evaluations // 3)
        elif pruner_method == 'none':
            # Do not prune
            pruner = MedianPruner(n_startup_trials=other_parameters.get('n_trials', 10), n_warmup_steps=n_evaluations)
        else:
            raise ValueError('Unknown pruner: {}'.format(pruner_method))

        logger.info ("Sampler: {} - Pruner: {}".format(sampler_method, pruner_method))

        #study = optuna.create_study(sampler=sampler, pruner=pruner)
        study_name = other_parameters.get('study_name', 'hp_study')  # Unique identifier of the study.
        study = optuna.create_study(study_name=study_name, storage='sqlite:///%s/%s.db' %(root_path, study_name), sampler=sampler, pruner=pruner, load_if_exists=True)

        def objective(trial):

            hp_parameters = parameters.copy()
            self.parameters_non_pickable = dict(trial=trial)

            if parameter_sampler is not None:
                hp_parameters.update(parameter_sampler(trial))

            if nruns is None:
                _, dict_results = self.create_experiment_and_run (parameters=hp_parameters, other_parameters = other_parameters, root_path=root_path, run_number=other_parameters.get('run_number'))
            else:
                mu_best, std_best, dict_results = self.run_multiple_repetitions (parameters=hp_parameters, other_parameters = other_parameters, root_path=root_path, nruns=nruns)

            if dict_results.get('is_pruned', False):
                raise optuna.structs.TrialPruned()

            return dict_results[other_parameters.get('key_score', 'cost')]

        optuna.logging.disable_propagation()
        study.optimize(objective, n_trials=other_parameters.get('n_trials', 10), n_jobs=other_parameters.get('n_jobs', 1))

        logger.info ('Number of finished trials: {}'.format(len(study.trials)))
        logger.info ('Best trial:')
        trial = study.best_trial
        logger.info ('Value: {}'.format(trial.value))
        logger.info ('best params: {}'.format (study.best_params))
        best_value = trial.value

        nruns_best = other_parameters.get('nruns_best', 0)
        if nruns_best > 0:
            logger.info ('running best configuration %d times' %nruns_best)
            parameters.update (study.best_params)
            mu_best, std_best, _ = self.run_multiple_repetitions (parameters=parameters, other_parameters = other_parameters,
                                            root_path=root_path, nruns=nruns_best)
            best_value = mu_best

        return best_value

    def rerun_experiment (self, experiments=[], run_numbers=[0], nruns = None, root_path=None, root_folder = None,
                          other_parameters={}, parameters = {}, parameter_sampler = None, parameters_multiple_values = None,
                          log_message='', only_if_exists=False):

        other_parameters = other_parameters.copy()

        if root_folder is not None:
            other_parameters['root_folder'] = root_folder

        if root_path is None:
            root_path = self.get_path_experiments(folder  = other_parameters.get('root_folder'))

        logger = set_logger ("experiment_manager", root_path)

        if nruns is not None:
            run_numbers = range (nruns)

        parameters_original = parameters
        other_parameters_original = other_parameters
        for experiment_id in experiments:
            check_experiment_matches = (parameters_multiple_values is None) and (parameter_sampler is None)
            parameters, other_parameters = load_parameters (experiment=experiment_id, root_path=root_path, root_folder = root_folder,
                                                            other_parameters=other_parameters_original, parameters = parameters_original,
                                                            check_experiment_matches=check_experiment_matches)

            # we need to set the following flag to False, since otherwise when we request to store the intermediate results
            # and the experiment did not start, we do not run the experiment
            if other_parameters.get('use_last_result', False) and not other_parameters_original.get('use_last_result', False):
                logger.debug ('changing other_parameters["use_last_result"] to False')
                other_parameters['use_last_result'] = False
            logger.info (f'running experiment {experiment_id} with parameters:\n{parameters}\nother_parameters:\n{other_parameters}')

            if parameter_sampler is not None:
                logger.info ('running hp_optimization')
                insert_experiment_script_path (other_parameters, logger)
                self.hp_optimization (parameter_sampler = parameter_sampler, root_path=root_path, log_message=log_message,
                                        parameters=parameters, other_parameters=other_parameters)
            elif parameters_multiple_values is not None:
                self.grid_search (parameters_multiple_values=parameters_multiple_values, parameters_single_value=parameters,
                                    other_parameters = other_parameters, root_path=root_path, run_numbers=run_numbers, log_message=log_message)
            else:
                if only_if_exists:
                    run_numbers = [run_number for run_number in run_numbers if os.path.exists('%s/%d' %(path_root_experiment, run_number))]

                script_parameters = {}
                insert_experiment_script_path (script_parameters, logger)
                other_parameters['rerun_script'] = script_parameters
                self.run_multiple_repetitions (parameters=parameters, other_parameters = other_parameters, root_path=root_path,
                                                log_message=log_message, run_numbers=run_numbers)

    def rerun_experiment_pipeline (self, experiments, run_numbers=None, root_path=None, root_folder=None, new_parameters={}, save_results=False):

        if root_path is None:
            root_path = self.get_path_experiments(folder=root_folder)
        for experiment_id in experiments:
            path_root_experiment = self.get_path_experiment (experiment_id, root_path=root_path)

            parameters, other_parameters=pickle.load(open('%s/parameters.pk' %path_root_experiment,'rb'))
            parameters = parameters.copy()
            parameters.update(other_parameters)
            parameters.update(new_parameters)
            for run_number in run_numbers:
                path_experiment = '%s/%d/' %(path_root_experiment, run_number)
                path_data = self.get_path_data (run_number, root_path, parameters)
                score, _ = self.run_experiment_pipeline (run_number, path_experiment, parameters = parameters)

                if save_results:
                    experiment_number = experiment_id
                    path_csv = '%s/experiments_data.csv' %root_path
                    path_pickle = path_csv.replace('csv', 'pk')
                    if os.path.exists(path_pickle):
                        experiment_data = pd.read_pickle (path_pickle)
                    else:
                        experiment_data = pd.read_csv (path_csv, index_col=0)
                    if type(score)==dict:
                        for key in score.keys():
                            if key != '':
                                experiment_data.loc[experiment_number, '%d_%s' %(run_number, key)]=score[key]
                            else:
                                experiment_data.loc[experiment_number, '%d' %run_number]=score[key]
                    else:
                        experiment_data.loc[experiment_number, name_score]=score
                    experiment_data.to_csv(path_csv)
                    experiment_data.to_pickle(path_pickle)

    def rerun_experiment_par (self, experiments, run_numbers=None, root_path=None, root_folder=None, parameters={}):

        if root_path is None:
            root_path = self.get_path_experiments(folder=root_folder)
        for experiment_id in experiments:
            path_root_experiment = self.get_path_experiment (experiment_id, root_path=root_path)

            for run_number in run_numbers:
                path_experiment = '%s/%d/' %(path_root_experiment, run_number)
                self.run_experiment_pipeline (run_number, path_experiment, parameters = parameters)

    def record_intermediate_results (self, experiments=range(100), run_numbers=range(100), root_path=None, root_folder=None, new_parameters={}, remove=False):

        if remove:
            new_parameters.update (remove_not_finished=True, only_remove_not_finished=True)
        else:
            new_parameters.update (use_last_result=True)

        self.rerun_experiment_and_save(experiments=experiments, run_numbers=run_numbers,
            root_path=root_path, root_folder=root_folder,
            new_parameters=new_parameters)
        
    def find_closest_epoch (self, experiment_data, parameters, name_epoch=dflt.name_epoch):
        '''Finds experiment with same parameters except for number of epochs, and takes the epochs that are closer but lower than the one in parameters.'''

        experiment_numbers, _, _ = experiment_utils.find_rows_with_parameters_dict (experiment_data, parameters, ignore_keys=[name_epoch,'prev_epoch'])

        defaults = self.get_default_parameters(parameters)
        current_epoch = parameters.get(name_epoch, defaults.get(name_epoch))
        if current_epoch is None:
            current_epoch = -1
        if len(experiment_numbers) > 1:
            epochs = experiment_data.loc[experiment_numbers,name_epoch].copy()
            epochs[epochs.isnull()]=defaults.get(name_epoch)
            epochs = epochs.loc[epochs<=current_epoch]
            if epochs.shape[0] == 0:
                return None
            else:
                return epochs.astype(int).idxmax()
        elif len(experiment_numbers) == 1:
            return experiment_numbers[0]
        else:
            return None
        
    def get_last_epoch (self, parameters, path_results, 
                             name_epoch=dflt.name_epoch, name_last_epoch=dflt.name_last_epoch):
        
        name_model_history = parameters.get('name_model_history', self.name_model_history)
        path_model_history = f'{path_results}/{name_model_history}'

        prev_epoch = -1
        if os.path.exists(path_model_history):
            summary = pickle.load(open(path_model_history, 'rb'))
            prev_epoch = summary.get(name_last_epoch)
            if prev_epoch is None:
                key_score = self.get_key_score (parameters)
                if key_score in summary and (isinstance(summary[key_score], list) 
                                             or isinstance(summary[key_score], np.array)):
                    prev_epoch = (~np.isnan(summary[key_score])).sum()

        return prev_epoch
        
    def finished_all_epochs (self, parameters, path_results, 
                             name_epoch=dflt.name_epoch, name_last_epoch=dflt.name_last_epoch):
        defaults = self.get_default_parameters (parameters)
        current_epoch = parameters.get(name_epoch, defaults.get(name_epoch))
        prev_epoch = self.get_last_epoch (parameters, path_results, name_epoch=name_epoch, 
                                          name_last_epoch=name_last_epoch)
        
        if prev_epoch >= current_epoch:
            finished = True
        else:
            finished = False

        return finished
    
    def make_resume_from_checkpoint (self, parameters, prev_path_results, use_best=False):

        if parameters.get('previous_model_file_name') is not None:
            previous_model_file_name = parameters['previous_model_file_name']
        else:
            model_extension = parameters.get('model_extension', 'h5')
            model_name = parameters.get('model_name', 'checkpoint_')
            epoch_offset = parameters.get('epoch_offset', 0)
            name_best_model = parameters.get('name_best_model', 'best_model')

        found = False
        name_model_history = parameters.get('name_model_history', 'model_history.pk')
        name_last_epoch = parameters.get('name_last_epoch', dflt.name_last_epoch)
        path_model_history = f'{prev_path_results}/{name_model_history}'
        if os.path.exists(path_model_history):
            parameters['resume_summary'] = path_model_history
            found = True
            parameters['prev_path_results'] = prev_path_results
            if parameters.get('previous_model_file_name') is not None:
                parameters['resume'] = f'{prev_path_results}/{previous_model_file_name}'
            elif use_best:
                parameters['resume'] = f'{prev_path_results}/{name_best_model}.{model_extension}'
            else:
                summary = pickle.load(open(path_model_history, 'rb'))
                prev_epoch = summary.get(name_last_epoch)
                key_score = self.get_key_score (parameters)
                if prev_epoch is None:
                    if key_score in summary and (isinstance(summary[key_score], list) 
                                                 or isinstance(summary[key_score], np.array)):
                        prev_epoch = (~np.isnan(summary[key_score])).sum()
                    else:
                        prev_epoch = 0

                if prev_epoch >= 0:
                    parameters['resume'] = f'{prev_path_results}/{model_name}{prev_epoch+epoch_offset}.{model_extension}'
            if not os.path.exists(parameters['resume']):
                path_resume2 = f'{prev_path_results}/{self.model_file_name}'
                if os.path.exists (path_resume2):
                    parameters['resume'] = path_resume2
                else:
                    parameters['resume'] = ''
                    parameters['prev_path_results'] = ''
                    found = False

        return found
    
    def exists_current_checkpoint (self, parameters, path_results):
        model_file_name = self.get_parameter (parameters, 'model_file_name')
        return os.path.exists (f'{path_results}/{model_file_name}')
    
    def get_parameter (self, parameters, key):
        parameter = parameters.get(key)
        return parameter if parameter is not None else getattr(self, key)
    
    def obtain_last_result (self, parameters, path_results):

        if parameters.get('use_last_result_from_dict', False):
            return self.obtain_last_result_from_dict (parameters, path_results)
        name_result_file = self.get_parameter (parameters, 'name_model_history')
        path_results_file = f'{path_results}/{name_result_file}'
        dict_results = None
        if os.path.exists (path_results_file):
            history = pickle.load(open(path_results_file, 'rb'))
            metrics = parameters.get('key_scores')
            if metrics is None:
                metrics = history.keys()
            ops = parameters.get('ops')
            if ops is None:
                ops = ['max'] * len(metrics)
            if type(ops) is str:
                ops = [ops] * len(metrics)
            if type(ops) is dict:
                ops_dict = ops
                ops = ['max'] * len(metrics)
                i = 0
                for k in metrics:
                    if k in ops_dict.keys():
                        ops[i] = ops_dict[k]
                    i += 1
            dict_results = {}
            max_last_position = -1
            for metric, op in zip(metrics, ops):
                if metric in history.keys():
                    history_array = history[metric]
                    score = min(history_array) if op == 'min' else max(history_array)
                    last_position = np.where(np.array(history_array).ravel()==0)[0]
                    if len(last_position) > 0:
                        last_position = last_position[0] - 1
                    else:
                        last_position = len(history_array)
                    dict_results[metric] = score
                else:
                    last_position = -1
                max_last_position = max(last_position, max_last_position)

            dict_results['last'] = max_last_position
            if max_last_position < parameters.get('min_iterations', dflt.min_iterations):
                dict_results = None
                print (f'not storing result from {path_results} with iterations {max_last_position}')
            else:
                print (f'storing result from {path_results} with iterations {max_last_position}')

        return dict_results
    
    #export
    def obtain_last_result_from_dict (self, parameters, path_results):
        name_result_file = self.get_parameter(parameters, 'result_file')
        path_results_file = '%s/%s' %(path_results, name_result_file)
        dict_results = None
        if os.path.exists (path_results_file):
            dict_results = pickle.load(open(path_results_file, 'rb'))
            if 'last' not in dict_results.keys() and 'epoch' in dict_results.keys():
                dict_results['last'] = dict_results['epoch']
            max_last_position = dict_results['last']
            if max_last_position < parameters.get('min_iterations', dflt.min_iterations):
                dict_results = None
                print ('not storing result from {} with iterations {}'.format(path_results, max_last_position))
            else:
                print ('storing result from {} with iterations {}'.format(path_results, max_last_position))

        return dict_results
    
    def register_and_store_subclassed_manager (self):
        #self.logger.debug ('registering')
        self.manager_factory.register_manager (self)
        self.manager_factory.write_manager (self)

### create_experiment_and_run

#### Setting up the experiment manager

In [5]:
#export tests.test_experiment_manager
def init_em (name_folder):
    em = ComplexDummyExperimentManager (path_experiments=f'test_{name_folder}')

    em.remove_previous_experiments()

    with pytest.raises (FileNotFoundError):
        os.listdir(em.get_path_experiments())
    
    return em

#### Basic usage

`create_experiment_and_run` is the main function of the `ExperimentManager`. All other functions make use of it adding additional functionalities.

In order to call `create_experiment_and_run`, we pass a dictionary of parameters characterizing the experiment we want to run, as follows:

In [6]:
#export tests.test_experiment_manager
def test_basic_usage ():
    em = init_em ('basic')
    
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    # The output is a tuple of two objects:
    #1. The main result metric. In our case, we didn't indicate the name of this metric, 
    #and therefore we get None.
    #1. A dictionary containing all the performance metrics for this experiment.
    
    assert result is None
    assert dict_results == {'validation_accuracy': 0.6, 'test_accuracy': 0.5}

    # Eight files  are stored in *path_experiments*, and the `experiments` folder is created:
    
    files_stored = ['current_experiment_number.pkl',
             'experiments',
             'experiments_data.csv',
             'experiments_data.pk',
             'git_hash.json',
             'other_parameters.csv',
             'parameters.pk',
             'parameters.txt',
             'summary.txt']
    
    display(files_stored)

    path_experiments = em.get_path_experiments()

    assert (sorted(os.listdir (path_experiments))==
            files_stored)

    # TODO TEST: test content of the above files
    
    import pandas as pd

    df = pd.read_pickle (f'{path_experiments}/experiments_data.pk')

    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()

    md ('experiment dataframe:'); display(df)
    
    list_exp = os.listdir (f'{path_experiments}/experiments')
    
    print (f'folder created in `{path_experiments}/experiments`:'); print(list_exp)
    
    assert list_exp == ['00000']
    
    print ('This folder has one sub-folder per run, since '
            'multiple runs can be done with the same parameters.')
    
    list_run = os.listdir (f'{path_experiments}/experiments/00000')
    
    print (f'contents of current run at `{path_experiments}/experiments/00000`:'); print(list_run)
    
    # the same data frame can be obtained by doing:
    df_bis = em.get_experiment_data ()
    
    pd.testing.assert_frame_equal(df,df_bis)
    
    em.remove_previous_experiments()

In [7]:
tst.run (test_basic_usage, tag='dummy')

script: /tmp/ipykernel_130431/725426095.py, line number: 5


running test_basic_usage
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


['current_experiment_number.pkl',
 'experiments',
 'experiments_data.csv',
 'experiments_data.pk',
 'git_hash.json',
 'other_parameters.csv',
 'parameters.pk',
 'parameters.txt',
 'summary.txt']

experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.001736,23:04:59.067385,True


folder created in `test_basic/experiments`:
['00000']
This folder has one sub-folder per run, since multiple runs can be done with the same parameters.
contents of current run at `test_basic/experiments/00000`:
['other_parameters.json', 'parameters.pk', 'parameters.txt', '0', 'parameters.json']


#### Running second experiment with same parameter values

In [8]:
#export tests.test_experiment_manager
def test_same_values ():
    em = init_em ('same_values')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
        
    em.raise_error_if_run = True
    # second experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    df = em.get_experiment_data ()

    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()

    md ('experiment dataframe:'); display(df)
    
    # As we can see, no new experiment is added to the DataFrame, since the values of the parameters used 
    # are already present in the first experiment.
    
    list_exp = os.listdir (f'{path_experiments}/experiments')
    
    print (f'folders created in `{path_experiments}/experiments`:'); print(list_exp)
    
    assert list_exp == ['00000']
    
    em.remove_previous_experiments()

In [9]:
tst.run (test_same_values, tag='dummy')

script: /tmp/ipykernel_130431/314616621.py, line number: 7


running test_same_values
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.002022,23:04:59.182501,True


folders created in `test_same_values/experiments`:
['00000']


#### Running second experiment with *almost* same parameter values

In [10]:
#export tests.test_experiment_manager
def test_almost_same_values ():
    em = init_em ('almost_same_values')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
        
    em.raise_error_if_run = True
    # second experiment: the difference between the values of rate parameter is 1.e-16: 
    # too small to be considered different
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05+1e-16})
    
    df = em.get_experiment_data ()
    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    list_exp = os.listdir (f'{path_experiments}/experiments')
    assert list_exp == ['00000']
    
    # consider 1.e-17 difference big enough
    em.raise_error_if_run = False
    # second experiment: the difference between the values of rate parameter is 1.e-16: 
    # too small to be considered different
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05+1e-16},
                                                        other_parameters={'precision': 1e-17})
    
    df = em.get_experiment_data ()
    assert df.shape[0]==2 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    display (df)
    list_exp = os.listdir (f'{path_experiments}/experiments')
    assert list_exp == ['00000', '00001']
    
    em.remove_previous_experiments()

In [11]:
tst.run (test_almost_same_values, tag='dummy', debug=False)

script: /tmp/ipykernel_130431/2249530880.py, line number: 7


running test_almost_same_values
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


script: /tmp/ipykernel_130431/2249530880.py, line number: 25


fitting model with 10 epochs
epoch 0: accuracy: 0.1500000000000001
epoch 1: accuracy: 0.2000000000000002
epoch 2: accuracy: 0.25000000000000033
epoch 3: accuracy: 0.30000000000000043
epoch 4: accuracy: 0.35000000000000053
epoch 5: accuracy: 0.40000000000000063
epoch 6: accuracy: 0.45000000000000073
epoch 7: accuracy: 0.5000000000000009
epoch 8: accuracy: 0.5500000000000009
epoch 9: accuracy: 0.600000000000001


,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.002137,23:04:59.294735,True
1,0.1,0.05,0.6,0.5,0.001731,23:04:59.331260,True


#### Adding new runs on previous experiment

In [12]:
#export tests.test_experiment_manager
def test_new_runs ():
    em = init_em ('new_runs')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
        
    # second experiment: in order to run another experiment with same parametres, we increase
    # the run number. The default run number used in the first experiment is 0, so we indicate
    # run_number=1
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05}, 
                                                         run_number=1)
    
    df = em.get_experiment_data ()

    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished', '1_validation_accuracy',
                                            '1_test_accuracy', 'time_1','1_finished']).all()

    md ('experiment dataframe:'); display(df)
    
    # another adding a new run number is to indicate run_number=None. This will make the experiment
    # manager find the next run number automatically. Since we have used run numbers 0 and 1, 
    # the next run number will be 2
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05}, 
                                                         run_number=None)
    
    df = em.get_experiment_data ()
    
    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished', '1_validation_accuracy',
                                            '1_test_accuracy', 'time_1','1_finished',
                                            '2_validation_accuracy', '2_test_accuracy', 'time_2', 
                                            '2_finished']).all()

    md ('experiment dataframe:'); display(df)
    
    # As we can see, no new experiment is added to the DataFrame, since the values of the parameters used 
    # are already present in the first experiment.
    
    list_exp = os.listdir (f'{path_experiments}/experiments')
    
    print (f'folders created in `{path_experiments}/experiments`:'); print(list_exp)
    
    assert list_exp == ['00000']
    
    list_runs = os.listdir (f'{path_experiments}/experiments/00000')
    assert sorted(list_runs) == ['0',
                                 '1',
                                 '2',
                                 'other_parameters.json',
                                 'parameters.json',
                                 'parameters.pk',
                                 'parameters.txt']
    
    em.remove_previous_experiments()

In [13]:
tst.run (test_new_runs, tag='dummy', debug=False)

script: /tmp/ipykernel_130431/3459580111.py, line number: 7


running test_new_runs
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


script: /tmp/ipykernel_130431/3459580111.py, line number: 13


fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


experiment dataframe:

...

script: /tmp/ipykernel_130431/3459580111.py, line number: 27


fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,1_validation_accuracy,1_test_accuracy,time_1,1_finished,2_validation_accuracy,2_test_accuracy,time_2,2_finished
0,0.1,0.05,0.6,0.5,0.001668,23:04:59.536757,True,0.6,0.5,0.002012,True,0.6,0.5,0.002238,True


folders created in `test_new_runs/experiments`:
['00000']


#### Adding second experiment

In [14]:
#export tests.test_experiment_manager
def test_second_experiment ():
    em = init_em ('second')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    md ('If we run a second experiment with new parameters, a new row is '
        'added to the dataframe, and a new folder is created:')
    
    # second experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.7, 'rate': 0.2})
    
    df = em.get_experiment_data ()

    assert df.shape[0]==2 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()

    md ('experiment dataframe:'); display(df)
    
    list_exp = os.listdir (f'{path_experiments}/experiments')
    
    md (f'folders created in `{path_experiments}/experiments`:'); print(list_exp)
    
    assert list_exp == ['00000','00001']
    
    em.remove_previous_experiments()

In [15]:
tst.run (test_second_experiment, tag='dummy')

script: /tmp/ipykernel_130431/3126725428.py, line number: 7


running test_second_experiment
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


If we run a second experiment with new parameters, a new row is added to the dataframe, and a new folder is created:

script: /tmp/ipykernel_130431/3126725428.py, line number: 13


fitting model with 10 epochs
epoch 0: accuracy: 0.8999999999999999
epoch 1: accuracy: 1.0999999999999999
epoch 2: accuracy: 1.2999999999999998
epoch 3: accuracy: 1.4999999999999998
epoch 4: accuracy: 1.6999999999999997
epoch 5: accuracy: 1.8999999999999997
epoch 6: accuracy: 2.0999999999999996
epoch 7: accuracy: 2.3
epoch 8: accuracy: 2.5
epoch 9: accuracy: 2.7


experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.001703,23:04:59.682563,True
1,0.7,0.20,1.0,1.0,0.001819,23:04:59.716228,True


folders created in `test_second/experiments`:

['00000', '00001']


#### Adding another parameter 

In [16]:
import pandas as pd
df = pd.DataFrame({'a':[1,2,3],'b':[1,None,3]})
df

,a,b
0,1,1.0
1,2,NaN
2,3,3.0


In [17]:
df.isna().loc[1,'b']

True

In [18]:
#export tests.test_experiment_manager
def test_new_parameter ():
    em = init_em ('another_parameter')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    # second experiment:
    # same parameters as before plus new parameter 'epochs' not indicated in first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5})
    
    df = em.get_experiment_data ()

    # a new experiment is added, and a new parameter `epochs` is added as additional column at the end
    assert df.shape[0]==2 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished','epochs']).all()
    
    assert (df.index==[0,1]).all()
    
    # the new parameter has None value for all previous experiments that did not indicated its value
    # In our case, the first experiment has None value for parameter `epochs`
    # This means that the default value of epochs is used for that parameter.
    # In our case, if we look at the implementation of DummyExperimentManager, we can see that 
    # the default value for epochs is 10.
    assert df.isna().loc[0,'epochs']
    
    assert df.loc[1,'epochs'] == 5.0

    md ('experiment dataframe:'); display(df)
    
    em.remove_previous_experiments()

In [19]:
tst.run (test_new_parameter, tag='dummy')

script: /tmp/ipykernel_130431/2342689092.py, line number: 7
script: /tmp/ipykernel_130431/2342689092.py, line number: 11


running test_new_parameter
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6
fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35


experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,epochs
0,0.1,0.05,0.60,0.50,0.001644,23:04:59.961083,True,None
1,0.1,0.05,0.35,0.45,0.001034,23:04:59.991625,True,5.0


#### Adding another parameter with default value

In [20]:
#export tests.test_experiment_manager
def test_new_parameter_default ():
    em = init_em ('another_parameter_default')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    # second experiment:
    # same parameters as before plus new parameter 'epochs' not indicated in first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 10})
    
    df = em.get_experiment_data ()

    # in this case, no new experiment is added, since the new parameter has the same value as the default value
    # implicitly used in the first experiment.
    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    
    assert (df.index==[0]).all()
    
    md ('experiment dataframe:'); display(df)
    
    em.remove_previous_experiments()

In [21]:
tst.run (test_new_parameter_default, tag='dummy')

script: /tmp/ipykernel_130431/2374034115.py, line number: 7


running test_new_parameter_default
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.001687,23:05:00.101218,True


#### Indicating parameters that don't affect the experiment

In [22]:
#export tests.test_experiment_manager
def test_other_parameters ():
    em = init_em ('other_parameters')
    path_experiments = em.get_path_experiments()
    
    # first experiment: 
    # we use the other_parameters argument to indicate a parameter that does not affect the outcome 
    # of the experiment
    # in this example, we change the level of verbosity. This parameter should not affect how the 
    # experiment runs, and therefore we tell our experiment manager to not create a new experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05},
                                                         other_parameters={'verbose': False})
    
    # second experiment:
    # same parameters as before except for the verbosity parameter. Our experiment manager considers
    # this experiment the same as before, and therefore it does not run it, but outputs the same results 
    # obtained before
    em.raise_error_if_run = True
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    df = em.get_experiment_data ()

    # in this case, no new experiment is added, since the new parameter has the same value as the default value
    # implicitly used in the first experiment.
    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    
    assert (df.index==[0]).all()
    
    md ('experiment dataframe:'); display(df)
    
    em.remove_previous_experiments()

In [23]:
tst.run (test_other_parameters, tag='dummy')

script: /tmp/ipykernel_130431/1875725149.py, line number: 12


running test_other_parameters


experiment dataframe:

,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.000246,23:05:00.207436,True


#### remove_not_finished

In order to use this functionality, we need to indicate the name of the parameter that specifies the number of epochs. This can be done either passing this when constructing the object:
```python
em = MyExperimentManager (name_epoch='epochs')
```
or indicating it in the `other_parameters` dictionary:
```python
other_parameters = dict(name_epoch='epochs', ...)
```

In [24]:
#export tests.test_experiment_manager
def test_remove_not_finished ():
    em = init_em ('remove_not_finished')
    path_experiments = em.get_path_experiments()
    
    # first experiment: we simulate that a halt before finishing
    with pytest.raises (KeyboardInterrupt):
        result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05},
                                                         other_parameters={'halt':True})
    
    df = em.get_experiment_data ()
    display(df)
    
    # second experiment: remove unfinished
    result, dict_results = em.create_experiment_and_run (parameters={'offset':1.0, 'rate': 0.2})
    
    df = em.get_experiment_data ()
    display(df)
    
    result, dict_results = em.create_experiment_and_run (parameters={'offset':1.0, 'rate': 0.3},
                                                         other_parameters={'remove_not_finished':True})
    
    df = em.get_experiment_data ()
    display(df)

    # in this case, no new experiment is added, since the new parameter has the same value as the default value
    # implicitly used in the first experiment.
    
    em.remove_previous_experiments()

In [25]:
tst.run (test_remove_not_finished, tag='dummy')

script: /tmp/ipykernel_130431/2801524373.py, line number: 9


running test_remove_not_finished
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


,offset,rate
0,0.1,0.05


script: /tmp/ipykernel_130431/2801524373.py, line number: 15


fitting model with 10 epochs
epoch 0: accuracy: 1.2
epoch 1: accuracy: 1.4
epoch 2: accuracy: 1.5999999999999999
epoch 3: accuracy: 1.7999999999999998
epoch 4: accuracy: 1.9999999999999998
epoch 5: accuracy: 2.1999999999999997
epoch 6: accuracy: 2.4
epoch 7: accuracy: 2.6
epoch 8: accuracy: 2.8000000000000003
epoch 9: accuracy: 3.0000000000000004


,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,NaN,NaN,NaN,NaN,NaN
1,1.0,0.20,1.0,1.0,0.001805,23:05:00.407541,True


script: /tmp/ipykernel_130431/2801524373.py, line number: 21


fitting model with 10 epochs
epoch 0: accuracy: 1.3
epoch 1: accuracy: 1.6
epoch 2: accuracy: 1.9000000000000001
epoch 3: accuracy: 2.2
epoch 4: accuracy: 2.5
epoch 5: accuracy: 2.8
epoch 6: accuracy: 3.0999999999999996
epoch 7: accuracy: 3.3999999999999995
epoch 8: accuracy: 3.6999999999999993
epoch 9: accuracy: 3.999999999999999


,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,NaN,NaN,NaN,NaN,NaN
1,1.0,0.20,1.0,1.0,0.001805,23:05:00.407541,True
2,1.0,0.30,1.0,1.0,0.001835,23:05:00.448265,True


#### repeat_experiment

In [26]:
#export tests.test_experiment_manager
def test_repeat_experiment ():
    em = init_em ('repeat_experiment')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    df = em.get_experiment_data ()
    display(df)
    date = df.date.values[0]
    
    # second experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05},
                                                         other_parameters={'repeat_experiment': True})
    
    df = em.get_experiment_data ()
    display(df)
    assert df.date.values[0] != date
    

    # in this case, no new experiment is added, since the new parameter has the same value as the default value
    # implicitly used in the first experiment.
    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    
    assert (df.index==[0]).all()
    
    em.remove_previous_experiments()

In [27]:
tst.run (test_repeat_experiment, tag='dummy', debug=False)

script: /tmp/ipykernel_130431/1962307353.py, line number: 7


running test_repeat_experiment
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.001832,23:05:00.564363,True


script: /tmp/ipykernel_130431/1962307353.py, line number: 15


fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,0.1,0.05,0.6,0.5,0.001832,23:05:00.600932,True


#### check_finished

In order to use this functionality, we need to indicate the name of the parameter that specifies the number of epochs. This can be done either passing this when constructing the object:
```python
em = MyExperimentManager (name_epoch='epochs')
```
or indicating it in the `other_parameters` dictionary:
```python
other_parameters = dict(name_epoch='epochs', ...)
```

In [28]:
#export tests.test_experiment_manager
def test_check_finished ():
    em = init_em ('check_finished')
    path_experiments = em.get_path_experiments()
    
    # first experiment: we simulate that we only run for half the number of epochs
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 10},
                                                         other_parameters={'actual_epochs': 5})
    
    df = em.get_experiment_data ()
    date = df.date.values[0]
    score = df['0_validation_accuracy'].values[0]
    
    # second experiment: same values in parameters dictionary, without other_parameters 
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 10})
    
    df = em.get_experiment_data ()
    
    assert (date==df.date.values[0]) and (score==df['0_validation_accuracy'].values[0])
    
    # third experiment: same values in parameters dictionary, with other_parameters indicating check_finished
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 10},
                                                         other_parameters={'check_finished':True})
    
    df = em.get_experiment_data ()
    assert df.shape[0]==1 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    assert (df.index==[0]).all()
    assert (date!=df.date.values[0]) and (score!=df['0_validation_accuracy'].values[0])
    
    em.remove_previous_experiments()

In [29]:
tst.run (test_check_finished, tag='dummy')

script: /tmp/ipykernel_130431/2482338563.py, line number: 8


running test_check_finished
fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35


script: /tmp/ipykernel_130431/2482338563.py, line number: 23


fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


#### recompute_metrics

In [30]:
#export tests.test_experiment_manager
def test_recompute_metrics ():
    em = init_em ('recompute_metrics')
    path_experiments = em.get_path_experiments()
    
    # first experiment
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05})
    
    df = em.get_experiment_data ()    
    # second experiment: new values 
    em.raise_error_if_run = True
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.02},
                                                         other_parameters={'recompute_metrics': True})
    
    df = em.get_experiment_data ()
    assert df.shape[0]==2 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()    
    assert np.isnan(df['0_validation_accuracy'].values[1])
    
    # third experiment: new values 
    em.raise_error_if_run = False
    result, dict_results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.02, 'epochs': 10},
                                                         other_parameters={'recompute_metrics':True,
                                                                           'force_recompute_metrics': True})
    
    df = em.get_experiment_data ()
    assert df.shape[0]==2 and (df.columns==['offset','rate','0_validation_accuracy','0_test_accuracy',
                                           'time_0', 'date', '0_finished']).all()
    assert (df.index==[0,1]).all()
    assert df['0_validation_accuracy'].values[1]==0.3
    
    em.remove_previous_experiments()

In [31]:
tst.run (test_recompute_metrics, tag='dummy', debug=False)

script: /tmp/ipykernel_130431/2340212191.py, line number: 7


running test_recompute_metrics
fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


script: /tmp/ipykernel_130431/2340212191.py, line number: 24


fitting model with 10 epochs
epoch 0: accuracy: 0.12000000000000001
epoch 1: accuracy: 0.14
epoch 2: accuracy: 0.16
epoch 3: accuracy: 0.18
epoch 4: accuracy: 0.19999999999999998
epoch 5: accuracy: 0.21999999999999997
epoch 6: accuracy: 0.23999999999999996
epoch 7: accuracy: 0.25999999999999995
epoch 8: accuracy: 0.27999999999999997
epoch 9: accuracy: 0.3


#### prev_epoch

In order to use this functionality, we need to indicate: 
1. The name of the parameter that specifies the number of epochs. 
1. The name of the file where the model is stored.
This can be done either passing this when constructing the object:
```python
em = MyExperimentManager (name_epoch='epochs', model_file_name='model_weights.pk')
```
or indicating it in the `other_parameters` dictionary:
```python
other_parameters = dict(name_epoch='epochs', model_file_name='model_weights.pk')
```

Furthermore, in order to work, we need our experiment manager to make use of the parameter `resume` or the parameter `prev_path_results`. In particular, we need it to load the model file whose path is indicated in `parameters['resume']`, or whose path is indicated in `f'{parameters["prev_path_results"]}/{self.model_file_name}'`

In [32]:
#export tests.test_experiment_manager
def test_prev_epoch ():
    em = init_em ('prev_epoch')

    # get reference result
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 17})
    reference_accuracy = em.model.accuracy
    reference_weight = em.model.weight
    df = em.get_experiment_data ()
    display (df)
    em.remove_previous_experiments()

    # first 3 experiments
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 10})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 20})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 15})

    # more epochs
    # in order to work, we need our experiment manager to make use of the 
    # parameter 'resume' or the parameter 'prev_path_results'. 
    # In particular, we need it to load the model file
    # whose path is indicated in parameters['resume'], or whose path is 
    # indicated in f'{parameters["prev_path_results"]}/{self.model_file_name}'
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 17},
                                      other_parameters={'prev_epoch': True})




    assert em.model.epochs==2 and em.model.current_epoch==17

    assert reference_accuracy==em.model.accuracy and reference_weight==em.model.weight

    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 17},
                                      other_parameters={'repeat_experiment': True})

    assert em.model.epochs==17 and em.model.current_epoch==17

    assert reference_accuracy==em.model.accuracy and reference_weight==em.model.weight

    em.remove_previous_experiments()

In [33]:
tst.run (test_prev_epoch, tag='dummy')

script: /tmp/ipykernel_130431/1621658181.py, line number: 6


running test_prev_epoch
fitting model with 17 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6
epoch 10: accuracy: 0.65
epoch 11: accuracy: 0.7000000000000001
epoch 12: accuracy: 0.7500000000000001
epoch 13: accuracy: 0.8000000000000002
epoch 14: accuracy: 0.8500000000000002
epoch 15: accuracy: 0.9000000000000002
epoch 16: accuracy: 0.9500000000000003


,epochs,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,17.0,0.1,0.05,0.95,0.85,0.002792,23:05:01.061801,True


script: /tmp/ipykernel_130431/1621658181.py, line number: 14


fitting model with 10 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6


script: /tmp/ipykernel_130431/1621658181.py, line number: 15
script: /tmp/ipykernel_130431/1621658181.py, line number: 16
script: /tmp/ipykernel_130431/1621658181.py, line number: 25
script: /tmp/ipykernel_130431/1621658181.py, line number: 35


fitting model with 20 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch 8: accuracy: 0.5499999999999999
epoch 9: accuracy: 0.6
epoch 10: accuracy: 0.65
epoch 11: accuracy: 0.7000000000000001
epoch 12: accuracy: 0.7500000000000001
epoch 13: accuracy: 0.8000000000000002
epoch 14: accuracy: 0.8500000000000002
epoch 15: accuracy: 0.9000000000000002
epoch 16: accuracy: 0.9500000000000003
epoch 17: accuracy: 1.0000000000000002
epoch 18: accuracy: 1.0500000000000003
epoch 19: accuracy: 1.1000000000000003
fitting model with 15 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
epoch 5: accuracy: 0.39999999999999997
epoch 6: accuracy: 0.44999999999999996
epoch 7: accuracy: 0.49999999999999994
epoch

In [60]:
#export tests.test_experiment_manager
def test_prev_epoch2 ():
    em = init_em ('prev_epoch2')
    
    em.remove_previous_experiments()
    score, results = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
                                          other_parameters={'actual_epochs': 2})
    
    
    assert score is None and results['validation_accuracy']==0.16
    assert em.model.current_epoch==2 and em.model.epochs==2
    
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.04, 'epochs': 5})

        
    # We use last result and have the required number of epochs to default number (50)
    # But we request to run the experiment until the end
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'prev_epoch': True, 'check_finished': True, 'use_previous_best': False}
    )

    assert score is None and results['validation_accuracy']==0.25
    assert em.model.current_epoch==5 and em.model.epochs==3
    df = em.get_experiment_data ()
    assert (df['0_validation_accuracy']==[0.25, 0.30]).all()
    

    em.remove_previous_experiments()

    # **********************************
    with pytest.raises (KeyboardInterrupt):
        _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
                                          other_parameters={'actual_epochs': 2, 'halt': True})
    
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.04, 'epochs': 5})
        
    # We use last result and have the required number of epochs to default number (50)
    # But we request to run the experiment until the end
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'prev_epoch': True, 'check_finished_if_interrupted': True, 
                          'use_previous_best': False}
    )
    assert score is None and results['validation_accuracy']==0.25
    assert em.model.current_epoch==5 and em.model.epochs==3
    df = em.get_experiment_data ()
    assert (df['0_validation_accuracy']==[0.25, 0.30]).all()
    em.remove_previous_experiments()

In [61]:
tst.run (test_prev_epoch2, tag='dummy', debug=False)

script: /tmp/ipykernel_130431/2837899774.py, line number: 7
script: /tmp/ipykernel_130431/2837899774.py, line number: 13
script: /tmp/ipykernel_130431/2837899774.py, line number: 20
script: /tmp/ipykernel_130431/2837899774.py, line number: 34
script: /tmp/ipykernel_130431/2837899774.py, line number: 36
script: /tmp/ipykernel_130431/2837899774.py, line number: 43


running test_prev_epoch2
fitting model with 2 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
fitting model with 5 epochs
epoch 0: accuracy: 0.14
epoch 1: accuracy: 0.18000000000000002
epoch 2: accuracy: 0.22000000000000003
epoch 3: accuracy: 0.26
epoch 4: accuracy: 0.3
reading model from test_prev_epoch2/experiments/00000/0/model_weights.pk
fitting model with 3 epochs
epoch 0: accuracy: 0.19
epoch 1: accuracy: 0.22
epoch 2: accuracy: 0.25
fitting model with 2 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
fitting model with 5 epochs
epoch 0: accuracy: 0.14
epoch 1: accuracy: 0.18000000000000002
epoch 2: accuracy: 0.22000000000000003
epoch 3: accuracy: 0.26
epoch 4: accuracy: 0.3
reading model from test_prev_epoch2/experiments/00000/0/model_weights.pk
fitting model with 3 epochs
epoch 0: accuracy: 0.19
epoch 1: accuracy: 0.22
epoch 2: accuracy: 0.25


#### from_exp

In order to work, we need our experiment manager to make use of the parameter `prev_path_results`. In particular, we need it to load the model file whose path is indicated in `f'{parameters["prev_path_results"]}/{self.model_file_name}'`

In [36]:
#export tests.test_experiment_manager
def test_from_exp ():
    em = init_em ('from_exp')
    path_experiments = em.get_path_experiments()

    # get reference result
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5})
    reference_accuracy = em.model.accuracy
    reference_weight = em.model.weight
    em.remove_previous_experiments()

    # first 3 experiments
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.03, 'epochs': 2})
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.04, 'epochs': 2})
    
    # the following resumes from experiment 0, and trains the model for 5 more epochs 
    # using now different `offset` and `rate` hyper-parameters
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.05, 'epochs': 5},
                                      other_parameters={'from_exp': 0})

    assert em.model.epochs==5 and em.model.current_epoch==7
    correct_accuracy = (0.1 + 0.03*2 # accuracy of model from experiment 0
                        + 0.05*5)     # accuracy gained by training for 5 more epochs using 
                                    #  new hyper-parameters: rate=0.05 
    assert (em.model.accuracy-correct_accuracy) < 1e-10
    assert reference_accuracy!=em.model.accuracy

    em.remove_previous_experiments()

In [37]:
tst.run (test_from_exp, tag='dummy')

script: /tmp/ipykernel_130431/3213666507.py, line number: 7
script: /tmp/ipykernel_130431/3213666507.py, line number: 13


running test_from_exp
fitting model with 5 epochs
epoch 0: accuracy: 0.15000000000000002
epoch 1: accuracy: 0.2
epoch 2: accuracy: 0.25
epoch 3: accuracy: 0.3
epoch 4: accuracy: 0.35
fitting model with 2 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16


script: /tmp/ipykernel_130431/3213666507.py, line number: 14
script: /tmp/ipykernel_130431/3213666507.py, line number: 19


fitting model with 2 epochs
epoch 0: accuracy: 0.14
epoch 1: accuracy: 0.18000000000000002
reading model from test_from_exp/experiments/00000/0/model_weights.pk
fitting model with 5 epochs
epoch 0: accuracy: 0.21000000000000002
epoch 1: accuracy: 0.26
epoch 2: accuracy: 0.31
epoch 3: accuracy: 0.36
epoch 4: accuracy: 0.41


#### skip_interrupted

In order to use this functionality, we need to indicate the name of the file where the model is stored.
This can be done either passing this when constructing the object:
```python
em = MyExperimentManager (model_file_name='model_weights.pk')
```
or indicating it in the `other_parameters` dictionary:
```python
other_parameters = dict(model_file_name='model_weights.pk')
```

Alternatively, we can indicate the name of the file where the model history exists. This can be done either passing this when constructing the object:
```python
em = MyExperimentManager (name_model_history='history.pk')
```
or indicating it in the `other_parameters` dictionary:
```python
other_parameters = dict(model_file_name='history.pk')
```
If not indicated, the experiment manager tries to find the model history in a file named `model_history.pk`. In order to consider the history good enough, the experiment manager checks if the length of the arrays stored in the model_history dictionary is at least `parameters.get('min_iterations', dflt.min_iterations)`

In [38]:
#export tests.test_experiment_manager
def test_skip_interrupted ():
    em = init_em ('skip_interrupted')
    path_experiments = em.get_path_experiments()

    # first 3 experiments
    with pytest.raises (KeyboardInterrupt):
        _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
                                          other_parameters={'halt': True})
    
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.04, 'epochs': 5})
    
    em.raise_error_if_run = True
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'skip_interrupted': True}
    )
    assert score==None and len(results)==0
    
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'skip_interrupted': True,
                          'model_file_name': 'wrong_file.pk',
                          'min_iterations':1}
    )
    assert score==None and len(results)==0
    
    with pytest.raises (RuntimeError):
        score, results = em.create_experiment_and_run (
            parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
            other_parameters={'skip_interrupted': True,
                              'model_file_name': 'wrong_file.pk'}
        )
    
    df = em.get_experiment_data ()
    display (df)

    em.remove_previous_experiments()

In [39]:
tst.run (test_skip_interrupted, tag='dummy', debug=False)

running test_skip_interrupted


script: /tmp/ipykernel_130431/4036359447.py, line number: 9
script: /tmp/ipykernel_130431/4036359447.py, line number: 11
script: /tmp/ipykernel_130431/4036359447.py, line number: 16


fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
fitting model with 5 epochs
epoch 0: accuracy: 0.14
epoch 1: accuracy: 0.18000000000000002
epoch 2: accuracy: 0.22000000000000003
epoch 3: accuracy: 0.26
epoch 4: accuracy: 0.3


script: /tmp/ipykernel_130431/4036359447.py, line number: 24
script: /tmp/ipykernel_130431/4036359447.py, line number: 32


storing result from test_skip_interrupted/experiments/00000/0 with iterations 5
not storing result from test_skip_interrupted/experiments/00000/0 with iterations 5


,epochs,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,5.0,0.1,0.03,NaN,NaN,NaN,NaN,NaN
1,5.0,0.1,0.04,0.3,0.4,0.001057,23:05:09.113386,True


#### use_last_result

In order to use this functionality, we need to indicate the name of the file where the model history exists. This can be done either passing this when constructing the object:
```python
em = MyExperimentManager (name_model_history='history.pk')
```
or indicating it in the `other_parameters` dictionary:
```python
other_parameters = dict(model_file_name='history.pk')
```
If not indicated, the experiment manager tries to find the model history in a file named `model_history.pk`. In order to consider the history good enough, the experiment manager checks if the length of the arrays stored in the model_history dictionary is at least `parameters.get('min_iterations', dflt.min_iterations)`

In [40]:
#export tests.test_experiment_manager
def test_use_last_result ():
    em = init_em ('use_last_result')
    path_experiments = em.get_path_experiments()

    # first 3 experiments
    with pytest.raises (KeyboardInterrupt):
        _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
                                          other_parameters={'halt': True})
    
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.04, 'epochs': 5})
    
    df = em.get_experiment_data ()
    assert (df.isna()['0_validation_accuracy'] == [True, False]).all()
    
    # We use last result but require that number of epochs is at least 50.
    # Since this is not true, the last result is not used.
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'use_last_result': True}
    )
    assert score==None and results=={}
    df = em.get_experiment_data ()
    display(df)
    assert (df.isna()['0_validation_accuracy'] == [True, False]).all()
    
    # We use last result and lower the required number of epochs to 2
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'use_last_result': True, 'min_iterations': 2}
    )
    print (score, results)
    assert score==None and results=={'validation_accuracy': 0.25, 'test_accuracy': 0.35, 'accuracy': 0.25, 'last': 5}
    df = em.get_experiment_data ()
    display(df)
    assert (df.isna()['0_validation_accuracy'] == [False, False]).all()
    assert (df['0_validation_accuracy'] == [0.25, 0.30]).all()
    
    # We use last result and increase the required number of epochs to default number (50)
    # But we request to run the experiment until the end
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'use_last_result': True, 'run_if_not_interrumpted': True}
    )
    print (score, results)
    #assert score==None and results=={'validation_accuracy': 0.25, 'test_accuracy': 0.35, 'accuracy': 0.25, 'last': 5}
    df = em.get_experiment_data ()
    display(df)
    #assert (df.isna()['0_validation_accuracy'] == [False, False]).all()
    #assert (df['0_validation_accuracy'] == [0.25, 0.30]).all()

    em.remove_previous_experiments()

In [41]:
tst.run (test_use_last_result, tag='dummy', debug=False)

running test_use_last_result


script: /tmp/ipykernel_130431/906478241.py, line number: 9


fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25


script: /tmp/ipykernel_130431/906478241.py, line number: 11
script: /tmp/ipykernel_130431/906478241.py, line number: 20


fitting model with 5 epochs
epoch 0: accuracy: 0.14
epoch 1: accuracy: 0.18000000000000002
epoch 2: accuracy: 0.22000000000000003
epoch 3: accuracy: 0.26
epoch 4: accuracy: 0.3
not storing result from test_use_last_result/experiments/00000/0 with iterations 5


...

script: /tmp/ipykernel_130431/906478241.py, line number: 30


storing result from test_use_last_result/experiments/00000/0 with iterations 5
None {'validation_accuracy': 0.25, 'test_accuracy': 0.35, 'accuracy': 0.25, 'last': 5}


,epochs,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,0_accuracy,0_last
0,5.0,0.1,0.03,0.25,0.35,NaN,23:05:09.360072,False,0.25,5.0
1,5.0,0.1,0.04,0.30,0.40,0.001131,23:05:09.297575,True,NaN,NaN


0.25 {'validation_accuracy': 0.25}


,epochs,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished,0_accuracy,0_last
0,5.0,0.1,0.03,0.25,0.35,NaN,23:05:09.360072,False,0.25,5.0
1,5.0,0.1,0.04,0.30,0.40,0.001131,23:05:09.297575,True,NaN,NaN


In [42]:
#export tests.test_experiment_manager
def test_use_last_result_run_interrupted ():
    em = init_em ('use_last_result_run_interrupted')
    path_experiments = em.get_path_experiments()

    # first 3 experiments
    with pytest.raises (KeyboardInterrupt):
        _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
                                          other_parameters={'actual_epochs': 2, 'halt': True})
    
    _ = em.create_experiment_and_run (parameters={'offset':0.1, 'rate': 0.04, 'epochs': 5})
    
    df = em.get_experiment_data ()
    #display (df)
    assert (df.isna()['0_validation_accuracy'] == [True, False]).all()
        
    # We use last result and have the required number of epochs to default number (50)
    # But we request to run the experiment until the end
    score, results = em.create_experiment_and_run (
        parameters={'offset':0.1, 'rate': 0.03, 'epochs': 5},
        other_parameters={'use_last_result': True, 'run_if_not_interrumpted': True}
    )
    print (score, results)
    #assert score==None and results=={'validation_accuracy': 0.25, 'test_accuracy': 0.35, 'accuracy': 0.25, 'last': 5}
    df = em.get_experiment_data ()
    #display(df)
    assert em.model.current_epoch==5 and em.model.epochs==5
    assert (df.isna()['0_validation_accuracy'] == [False, False]).all()
    assert (df['0_validation_accuracy'] == [0.25, 0.30]).all()

In [43]:
tst.run (test_use_last_result_run_interrupted, tag='dummy', debug=False)

running test_use_last_result_run_interrupted


script: /tmp/ipykernel_130431/2429739771.py, line number: 9


fitting model with 2 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16


script: /tmp/ipykernel_130431/2429739771.py, line number: 11
script: /tmp/ipykernel_130431/2429739771.py, line number: 21


fitting model with 5 epochs
epoch 0: accuracy: 0.14
epoch 1: accuracy: 0.18000000000000002
epoch 2: accuracy: 0.22000000000000003
epoch 3: accuracy: 0.26
epoch 4: accuracy: 0.3
not storing result from test_use_last_result_run_interrupted/experiments/00000/0 with iterations 2
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
None {'validation_accuracy': 0.25, 'test_accuracy': 0.35}


In [44]:
#%debug

## get_git_revision_hash

In [45]:
#export
def get_git_revision_hash (root_path=None):
    try:
        git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'])
        git_hash = str(git_hash)
        json.dump(git_hash, open('%s/git_hash.json' %root_path, 'wt'))
    except:
        logger = logging.getLogger("experiment_manager")
        if root_path is not None:
            logger.info ('could not get git hash, retrieving it from disk...')
            git_hash = json.load(open('%s/git_hash.json' %root_path, 'rt'))
        else:
            logger.info ('could not get git hash, using empty string...')
            git_hash = ''

    return str(git_hash)

## record_parameters

In [46]:
#export
def record_parameters (path_save, parameters, other_parameters=None):
    with open('%s/parameters.txt' %path_save, 'wt') as f:
        f.write('%s\n' %mypprint(parameters, dict_name='parameters'))
        if other_parameters is not None:
            f.write('\n\n%s\n' %mypprint(other_parameters, dict_name='other_parameters'))
    if other_parameters is not None:
        pickle.dump ([parameters,other_parameters],open('%s/parameters.pk' %path_save, 'wb'))
    else:
        pickle.dump (parameters,open('%s/parameters.pk' %path_save, 'wb'))
    try:
        json.dump(parameters,open('%s/parameters.json' %path_save, 'wt'))
    except:
        pass
    if other_parameters is not None:
        try:
            json.dump(parameters,open('%s/other_parameters.json' %path_save, 'wt'))
        except:
            pass

## mypprint

In [47]:
#export
def mypprint(parameters, dict_name=None):
    if dict_name is not None:
        text = '%s=dict(' %dict_name
        tpad = ' ' * len(text)
    else:
        text = '\t'
        tpad = '\t'
    for idx, (key, value) in enumerate(sorted(parameters.items(), key=lambda x: x[0])):
        if type(value) is str:
            value = '%s%s%s' %("'",value,"'")
        text += '{}={}'.format(key, value)
        if idx < (len(parameters)-1):
            text += ',\n{}'.format(tpad)

    if dict_name is not None:
        text += ')\n'
    else:
        text += '\n'

    return text

## mymakedirs

In [48]:
#export
def mymakedirs (path, exist_ok=False):
    '''work around for python 2.7'''
    if exist_ok:
        try:
            os.makedirs(path)
        except:
            pass
    else:
        os.makedirs(path)

## load_or_create_experiment_values

In [49]:
#export
def load_or_create_experiment_values (path_csv, parameters, precision=1e-15):

    logger = logging.getLogger("experiment_manager")
    path_pickle = path_csv.replace('csv', 'pk')
    experiment_numbers = []
    changed_dataframe = False

    if os.path.exists (path_pickle) or os.path.exists (path_csv):
        if os.path.exists (path_pickle):
            experiment_data = pd.read_pickle (path_pickle)
        else:
            experiment_data = pd.read_csv (path_csv, index_col=0)
            experiment_data.to_pickle(path_pickle)

        experiment_data, removed_defaults = remove_defaults_from_experiment_data (experiment_data)

        # Finds rows that match parameters. If the dataframe doesn't have any parameter with that name, a new column is created and changed_dataframe is set to True
        experiment_numbers, changed_dataframe, _ = experiment_utils.find_rows_with_parameters_dict (
            experiment_data, parameters, precision = precision
        )

        changed_dataframe = changed_dataframe or removed_defaults

        if len(experiment_numbers) > 1:
            logger.info ('more than one matching experiment: ', experiment_numbers)
    else:
        experiment_data = pd.DataFrame()

    if len(experiment_numbers) == 0:
        experiment_data = experiment_data.append (parameters, ignore_index=True)
        changed_dataframe = True
        experiment_number = experiment_data.shape[0]-1
    else:
        experiment_number = experiment_numbers[0]

    if changed_dataframe:
        experiment_data.to_csv(path_csv)
        experiment_data.to_pickle(path_pickle)

    return experiment_number, experiment_data

## store_parameters

In [50]:
#export
def store_parameters (root_path, experiment_number, parameters):
    """ Keeps track of dictionary to map experiment number and parameters values for the different experiments."""
    path_hp_dictionary = '%s/parameters.pk' %root_path
    if os.path.exists(path_hp_dictionary):
        all_parameters = pickle.load (open(path_hp_dictionary,'rb'))
    else:
        all_parameters = {}
    if experiment_number not in all_parameters.keys():
        str_par = '\n\nExperiment %d => parameters: \n%s\n' %(experiment_number,mypprint(parameters))
        f = open('%s/parameters.txt' %root_path, 'at')
        f.write(str_par)
        f.close()
        all_parameters[experiment_number] = parameters
        pickle.dump (all_parameters, open(path_hp_dictionary,'wb'))

    # pickle number of current experiment, for visualization
    pickle.dump(experiment_number, open('%s/current_experiment_number.pkl' %root_path,'wb'))

## isnull

In [51]:
#export
def isnull (experiment_data, experiment_number, name_column):
    return (name_column not in experiment_data.columns) or (experiment_data.loc[experiment_number, name_column] is None) or np.isnan(experiment_data.loc[experiment_number, name_column])

## get_experiment_number

In [52]:
#export
def get_experiment_number (root_path, parameters = {}):

    path_csv = '%s/experiments_data.csv' %root_path
    path_pickle = path_csv.replace('csv', 'pk')
    experiment_number, _ = load_or_create_experiment_values (path_csv, parameters)

    return experiment_number

## get_experiment_numbers

In [53]:
#export
def get_experiment_numbers (path_results_base, parameters_single_value, parameters_multiple_values_all):

    experiment_numbers = []

    parameters_multiple_values_all = list(ParameterGrid(parameters_multiple_values_all))

    for (i_hp, parameters_multiple_values) in enumerate(parameters_multiple_values_all):
        parameters = parameters_multiple_values.copy()
        parameters.update(parameters_single_value)
        parameters = remove_defaults (parameters)

        experiment_number = get_experiment_number (path_results_base, parameters=parameters)
        experiment_numbers.append(experiment_number)

    return experiment_numbers

## set_logger

In [54]:
#export
def set_logger (name, path_results, stdout=True, mode='a', just_message = False, filename='logs.txt'):

    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    for hdlr in logger.handlers[:]:  # remove all old handlers
        logger.removeHandler(hdlr)

    #if not logger.hasHandlers():

    # Create handlers
    if stdout:
        c_handler = logging.StreamHandler()
        c_handler.setLevel(logging.DEBUG)
        c_format = logging.Formatter('%(message)s')
        c_handler.setFormatter(c_format)
        logger.addHandler(c_handler)

    f_handler = logging.FileHandler('%s/%s' %(path_results, filename), mode = mode)
    f_handler.setLevel(logging.DEBUG)
    if just_message:
        f_format = logging.Formatter('%(asctime)s - %(message)s')
    else:
        f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    f_handler.setFormatter(f_format)
    logger.addHandler(f_handler)
    logger.propagate = 0

    return logger

## insert_experiment_script_path

In [55]:
#export
def insert_experiment_script_path (other_parameters, logger):
    if other_parameters.get('script_path') is None:
        stack_level = other_parameters.get('stack_level', -3)
        stack = traceback.extract_stack()[stack_level]
        other_parameters['script_path'] = stack.filename
        other_parameters['lineno'] = stack.lineno
        logger.info ('experiment script: {}, line: {}'.format(stack.filename, stack.lineno))
        if 'stack_level' in other_parameters:
            del other_parameters['stack_level']

## load_parameters

In [56]:
#export
def load_parameters (experiment=None, root_path=None, root_folder = None, other_parameters={}, parameters = {}, check_experiment_matches=True):

    from hpsearch.config.hpconfig import get_path_experiments, get_path_experiment
    if root_folder is not None:
        other_parameters['root_folder'] = root_folder

    if root_path is None:
        root_path = get_path_experiments(folder  = other_parameters.get('root_folder'))

    path_root_experiment = get_path_experiment (experiment, root_path=root_path)

    logger = set_logger ("experiment_manager", root_path)

    if os.path.exists('%s/parameters.pk' %path_root_experiment):
        parameters2, other_parameters2=pickle.load(open('%s/parameters.pk' %path_root_experiment,'rb'))

        other_parameters2.update(other_parameters)
        other_parameters = other_parameters2

        # if we don't add or modify parameters, we require that the old experiment number matches the new one
        if (len(parameters) == 0) and check_experiment_matches:
            logger.info ('requiring experiment number to be {}'.format(experiment))
            other_parameters['experiment_number'] = experiment
        elif 'experiment_number' in other_parameters:
            del other_parameters['experiment_number']

        parameters2.update(parameters)
        parameters = parameters2
    else:
        raise FileNotFoundError ('file {} not found'.format ('%s/parameters.pk' %path_root_experiment))

    return parameters, other_parameters

## save_other_parameters

In [57]:
#export
def save_other_parameters (experiment_number, other_parameters, root_path):
    parameters_to_save = {}
    for k in other_parameters.keys():
        if type(other_parameters[k]) is str:
            parameters_to_save[k] = other_parameters[k]
        elif np.isscalar(other_parameters[k]) and np.isreal(other_parameters[k]):
            parameters_to_save[k] = other_parameters[k]

    path_csv = '%s/other_parameters.csv' %root_path
    df = pd.DataFrame (index = [experiment_number], data=parameters_to_save)

    if os.path.exists (path_csv):
        df_all = pd.read_csv (path_csv, index_col=0)
        df_all = pd.concat([df_all, df], sort=True)
        df_all = df_all.loc[~df_all.index.duplicated(keep='last')]
    else:
        df_all = df
    df_all.to_csv (path_csv)